# Imports and Setups

In [1]:
! pip install transformers==4.10.1

     |████████████████████████████████| 2.8 MB 7.2 MB/s 
     |████████████████████████████████| 596 kB 68.5 MB/s 
     |████████████████████████████████| 3.3 MB 39.8 MB/s 
     |████████████████████████████████| 880 kB 52.9 MB/s 
     |████████████████████████████████| 77 kB 4.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=b518637ac5292272031bf17069e0a77b9478c70fea7c3b250e78e2c88dc742da
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
! pip install nlpaug==1.1.7

     |████████████████████████████████| 405 kB 8.1 MB/s 


In [3]:
! pip install sentencepiece

     |████████████████████████████████| 1.2 MB 9.4 MB/s 


In [4]:
!pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 4.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=5ef8a6e15014fb5ef2f9f5d391d7a91c5c56fb51530b0d892eef79c6ec416644
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [5]:
! pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git

  Cloning https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to /tmp/pip-req-build-ch8ux4im
  Running command git clone -q https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git /tmp/pip-req-build-ch8ux4im
     |████████████████████████████████| 50 kB 3.2 MB/s 
  Created wheel for parrot: filename=parrot-1.0-py3-none-any.whl size=8695 sha256=9ab9735ac55edf025f0e72f3b08819af3331b5eed333c24370cd020a2a3201c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-n9_kezkh/wheels/ff/27/79/a53e92f32da44c8c4f51094ca156e6ff91103b7e48a341d62b
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149852 sha256=38ed5000c5df3dda6d62889e503ea9b63fe3b2edec8c832468eb4974bfe5619d
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built parrot python-Levenshtein


Lendo o Arquivo

In [6]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [7]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


In [41]:
num_gen = 5
perc = 100

Gerando Base Percentual

In [42]:
arq = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_trec_'+str(perc)+'_original.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [43]:
base_cru_1p.head()

,texto,label
0,How did serfdom develop in and then leave Russ...,0
1,What films featured the character Popeye Doyle ?,1
2,How can I find a list of celebrities ' real na...,0
3,What fowl grabs the spotlight after the Chines...,1
4,What is the full form of .com ?,2


In [44]:
base_cru_1p.shape

(5452, 2)

Só rodar quando gerar o original

In [12]:
repeticoes = 1

In [ ]:
# arq = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_original.csv'
# base_cru_1p.to_csv(arq, encoding='utf-8', index=False, sep=';', decimal = ',')

In [13]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf

from nlpaug.util import Action

# Embedding Word2Vec


In [14]:
from nlpaug.util.file.download import DownloadUtil
DownloadUtil.download_glove('glove.6B', '.')

In [15]:
aug_emb = naf.Sequential([naf.Sometimes([
    naw.WordEmbsAug(
    model_type='glove', model_path='./glove.6B.200d.txt',
    action="substitute")]),
    naf.Sometimes([naw.WordEmbsAug(
    model_type='glove', model_path='./glove.6B.200d.txt',
    action="insert"),
])])

In [45]:
exp_emb = []

for i in range(repeticoes):
  base_cru_1p_emb = base_cru_1p.copy()
  base_cru_1p_emb['parafraseador'] = base_cru_1p_emb['texto'].progress_apply(lambda x:aug_emb.augment(x,num_gen))
  base_cru_1p_emb = base_cru_1p_emb.explode('parafraseador').reset_index(drop=True)
  exp_emb.append(base_cru_1p_emb)

  arquivo = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_trec_'+str(perc)+'_emb'+str(i+1)+'.csv'
  exp_emb[i].to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

100%|██████████| 5452/5452 [1:49:22<00:00,  1.20s/it]
